In [348]:
import yfinance as yf
import pandas as pd
import numpy as np
from autograd import grad
import altair as alt

#Solana (SOL-USD)
dados = yf.download('SOL-USD', start='2020-04-10', end='2024-10-21', interval='1d')
dados.reset_index(inplace=True)

dados = dados[['Date', 'Close']]

dados.tail()


[*********************100%***********************]  1 of 1 completed


Price,Date,Close
Ticker,,SOL-USD
1650,2024-10-16 00:00:00+00:00,154.234222
1651,2024-10-17 00:00:00+00:00,150.316025
1652,2024-10-18 00:00:00+00:00,154.975937
1653,2024-10-19 00:00:00+00:00,159.639359
1654,2024-10-20 00:00:00+00:00,166.939819


In [349]:
def porcentagem(dados):
    setenta = int(0.7 * len(dados))
    treino = dados.iloc[:setenta]
    teste = dados.iloc[setenta:]
    return treino, teste

treino, teste = porcentagem(dados)
print(treino.head())
print(teste.head())

Price                       Date     Close
Ticker                             SOL-USD
0      2020-04-10 00:00:00+00:00  0.951054
1      2020-04-11 00:00:00+00:00  0.776819
2      2020-04-12 00:00:00+00:00  0.882507
3      2020-04-13 00:00:00+00:00  0.777832
4      2020-04-14 00:00:00+00:00  0.661925
Price                       Date      Close
Ticker                              SOL-USD
1158   2023-06-12 00:00:00+00:00  15.196708
1159   2023-06-13 00:00:00+00:00  15.018193
1160   2023-06-14 00:00:00+00:00  14.486572
1161   2023-06-15 00:00:00+00:00  14.759473
1162   2023-06-16 00:00:00+00:00  15.308254


In [350]:
def criar_janelas(data, k=5):
    X, y = [], []
    for i in range(len(data) - k):
        X.append(data[i:i + k])
        y.append(data[i + k])
    return np.array(X), np.array(y)

X_treino, y_treino = criar_janelas(treino['Close'].values)
X_teste, y_teste = criar_janelas(teste['Close'].values)

In [351]:
def preparar_dados_para_treino(X):
    X = X.reshape(X.shape[0], -1)
    return np.hstack([X, np.ones((X.shape[0], 1))])

X_treino_bias = preparar_dados_para_treino(X_treino)
X_teste_bias = preparar_dados_para_treino(X_teste)

In [352]:
def mse_loss(w, X, y):
    y_pred = X @ w
    erro = y_pred - y
    return (erro ** 2).sum() / len(y)

gradiente_mse = grad(mse_loss)

def treinar_modelo(X, y):
    w = np.zeros((X.shape[1], 1))

    for i in range(1000):
        grad_w = gradiente_mse(w, X, y)
        w -= grad_w * 0.000006

    return w

w_otimo = treinar_modelo(X_treino_bias, y_treino)

y_pred = X_teste_bias @ w_otimo


In [353]:
df_resultados = pd.DataFrame({
    'Amostra': range(len(y_teste)),
    'Valor Real': y_teste.flatten(),
    'Previsão': y_pred.flatten()
})
df_resultados_melted = df_resultados.melt('Amostra', var_name='Tipo', value_name='Preço')

In [354]:
grafico = alt.Chart(df_resultados_melted).mark_line().encode(
    x='Amostra:Q',
    y='Preço:Q',
    color='Tipo:N'
).properties(
    title='Previsão vs Valores Reais',
    width=800,
    height=400
)

grafico.show()


alt.Chart(...)

In [355]:
def treinar_e_avaliar(dados, nome_escala):
    treino, teste = porcentagem(dados)

    X_treino, y_treino = criar_janelas(treino['Close'].values)
    X_teste, y_teste = criar_janelas(teste['Close'].values)

    X_treino_bias = preparar_dados_para_treino(X_treino)
    X_teste_bias = preparar_dados_para_treino(X_teste)

    w_otimo = treinar_modelo(X_treino_bias, y_treino)

    y_pred = X_teste_bias @ w_otimo

    df_resultados = pd.DataFrame({
        'Amostra': range(len(y_teste)),
        'Valor Real': y_teste.flatten(),
        'Previsão': y_pred.flatten()
    })
    df_resultados_melted = df_resultados.melt('Amostra', var_name='Tipo', value_name='Preço')

    grafico = alt.Chart(df_resultados_melted).mark_line().encode(
        x='Amostra:Q',
        y='Preço:Q',
        color='Tipo:N'
    ).properties(
        title=f'Previsão vs Valores Reais ({nome_escala})',
        width=800,
        height=400
    )
    grafico.show()


In [356]:
dados_semanais = yf.download('SOL-USD', start='2020-04-10', end='2024-10-21', interval='1wk')
dados_semanais.reset_index(inplace=True)

dados_semanais = dados_semanais[['Date', 'Close']]

treinar_e_avaliar(dados_semanais, "Semanal")

dados_mensais = yf.download('SOL-USD', start='2020-04-10', end='2024-10-21', interval='1mo')
dados_mensais.reset_index(inplace=True)

dados_mensais = dados_mensais[['Date', 'Close']]

treinar_e_avaliar(dados_mensais, "Mensal")

[*********************100%***********************]  1 of 1 completed

alt.Chart(...)

[*********************100%***********************]  1 of 1 completed


alt.Chart(...)